# PLacement Optimization of Public EV Charging Stations

*Note: Data to run this executable has been removed, to recreate the same datasets, please request access from the 99P Labs Developer Portal and use the code modules to generate them*

## Data Acquisition and Wrangling

### kotoTelematics

We built a module named `kotoTelematics`, that instantiates a connection the the 99P Vehicle Telematics data API.    

*Any class methods, static methods, general functions from this module cannot be executed by the TA/grader, as it requires an API access_token provided by Honda R&D's mobility research group, 99PLabs. We will symbolically outline their usage below and then make available the processed data that is generated as a result.*

In [ ]:
#Import module
import kotoTelematics as kt
#Instantiate telematics class with access key
#Not executable, requires access_token approved and provided by Honda R&D Mobility Research group
kt_obj = kt.telematics(access_token="my_99plabs_devloper_portal_access_token")

### telematics class object

The module contains, three main class methods, `vin_table_request`, `ts_table_request`, and `ts_table_scrape`. We found the API service had some throughput limitations on how much data we could request, so we primarily focused on the `ts_table_scrape`.  
This method calls the ts_table_request method, in a looping fashion, nudging the timewindow parameters along, to essentially scrape every record in the API

In [ ]:
#This is an example of what a single call to the API looks like
    #See ts_table_request method doc string in kotoTelematics.py module for more detailed info
columns = ['vin', 'navigation_location_coordinate_longitude', 'navigation_location_coordinate_latitude', 'sequence']
ts_response_object = kt_obj.ts_table_request(fetch_size=3, columns=columns, min_date='2020-01-12', max_date='2020-01-13')

### ts_table_scrape

The `ts_table_scrape` function iteratively exposes the data from the API in chunks that are <= **fetchsize** and span the length of the **start_date**, **start_time** + **time_window**(mins). The final argument to this method is a function parameter. This parameter gets passed a static method from within the class and is referred to as a query function. It is what each response from the API is passed to, and performs some logic and aggregation against it.

In [ ]:
#Example Using ts_table_scrape with demo function
    #See doc string in kotoTelematics.py module for more detailed info
ts_scrape_response_object = kt_obj.ts_table_scrape(\
    #fetch_size specifies how many rows to return, we found 150k was manageable by the API capacity, our local workspaces, and for a
    #10 minute window it was able to always pull all the available data
    fetch_size=150000, \
    #n_loops specifies how many times the window should be pushed forward
    n_loops=3000, \
    columns=columns, \
    #This specifies the start_date of the time window to be called
    start_date="2020-01-10", \
    #This specifies the start hour:minute:second of the time window to be called
    start_time="00:00:00", \
    #This specifies the range of the time window to be called in minutes
    time_window=10, \
    #This applies a query function static method to every response
    functions=[*QUERY FUNCTION GOES HERE*])

### destination_collector and data handling

The `destination_collector` is a query function. It takes each response from the API and creates a nested dictionary object where the outer most key is a vin and it's value is a dictionary. The inner key is a trip sequence number (which is unique when paired with vin), and the value of the inner dictionary is an array that stores latitude and longitude. Within one iteration response, the `destination_collector` continously updates the inner dictionary value, and because the data is event based and is returned arranged by timestamp, the last destination value that it updates in the array is representative of the last known location of that vehicle for that trip sequence during that window of time. This data is then picked up by the `dc_mongo_handler` function which is referred to as a data handler. It takes the dictionary that represents the query function output for one API window response, and stores it in a local MongoDB document database, where the id is the vin, and the document is the inner dictionary. As each new chunk comes in the `dc_mongo_handler` does the same thing, making the required logical comparisons and then updates the location for any vehicle and trip sequence combinations. When a new trip sequence starts it begins updating the values for that key, and what we are left with is the final transmitted location from every vehicle for every trip. AKA, the **destination**. For additional sharing capabilities, and adding more features to the modules, we also implemented a data handler for S3 buckets and local SQLite. They performed the comparison logic and updates in a similar fashion as the mongo handler. 

In [ ]:
#Example of calling table scraping method, with destination_collector supplied as the query function.
#destination_collector internally either uses the dc_mongo_handler, the dc_s3, or dc_sqlite handler.
ts_scrape_response_object = kt_obj.ts_table_scrape(fetch_size = 150000, n_loops=3000, columns=columns, start_date="2020-01-10", start_time="00:00:00", time_window=10, functions=[kt_obj.destination_collector])

### dwell_time_collector and data handling

The `dwell_time_collector` is a query function. It takes each response from the API and creates a nested dictionary object where the outer most key is a vin and its value is a dictionary. The inner key is a trip sequence number (which is unique when paired with vin), and the value of the inner dictionary is a 1x2 array where the first value is the first observed timestamp from that trip sequence, and the second value is any timestamp observed from that vehicle and trip sequence so long as it is greater than the value in the first position. This query function also has an accompanying data handler function, `dtc_mongo_handler`, which makes the necessary logical comparisons and preserves the 1st position in the array, AKA trip start time, and continously updates the second position in the array. This leads to a data structure, where the second position in the array for each vehicle + trip sequence combination represents the last observed timestamp, AKA trip end time. For additional sharing capabilites, we implemented similar handlers for storing in an S3 bucket and local SQLite. 

In [ ]:
#Example of calling table scraping method, with dwell_time_collector supplied as the query function.
#dwell_time_collector internally either uses the dtc_mongo_handler, the dtc_s3, or dtc_sqlite handler.
ts_scrape_response_object = kt_obj.ts_table_scrape(fetch_size = 150000, n_loops=1000, columns=columns, start_date="2020-01-12", start_time="00:00:00", time_window=10, functions=[kt_obj.dwell_time_collector])

### kotoMongoUtils

We built a module named `kotoMongoUtils`, that instantiates a connection to our local MongoDB instance, and creates a reference a specific collection within a document database. This module has a method for the destination collection called `mongo_dc_to_pd` and `mongo_dtc_to_pd`, which first create a list of all vins that exist in the respective collections, then uses that as an iterable, to get every document attached to every vin, while flattening it and appending it to a dataframe.  

*This class, and these methods cannot be executed by a TA/Grader because it references a local instance of a MongoDB that we were using as intermediate storage. However, we will include the dataframe as a flat file that the methods output.*

In [ ]:
#Import module
import kotoMongoUtils as ktmu
#Instantiate mongo class with client, db and collection
ktmu_obj = ktmu.mongo(client="mongodb://localhost:27017", db="kotomatic_db", collection="destinationCollector")
#Mongo_dc_to_pd retrieves all data stored in the destination collection and wrangles it into an appropriately structured dataframe
dest_raw_df = ktmu_obj.mongo_dc_to_pd()
#Mongo_dtc_to_pd retrieves all data stored in the dwell collection and wrangles it into an appropriately structure dataframe
dwell_raw_df = ktmu_obj.mongo_dtc_to_pd()

## Candidate Location Identification

### kotoDensityScan

We built a module named `kotoDensityScan`, that instantiates an implementation of an unsupervised learning algorithm from sci-kit learn called: DBSCAN (Density Based Spatial Clustering Applications w/ Noise). We added methods to fit the model, and trim the data to only the city of interest. *See Final Report for more details*

In [786]:
import pandas as pd
#Import the flat file of the mongo destination collection storing vin, sequence, destinations
dest_raw_df = pd.read_csv("data/dest_raw_df.csv", index_col=[0])
#Import the flat file of the mongo dwell time collection storing vin, sequence, start and end timestamp
dwell_raw_df = pd.read_csv("data/dwell_raw_df.csv", index_col=[0])

In [787]:
#Import module
import density_scan as dens
#Instantiate density_cluster class with data frame, and coordinate columns specified
dens_obj = dens.density_cluster(data=dest_raw_df,lat="Lat",lon="Lon")

#Fence Data is a method that removes all that isn't inside the square abritrary geofence we drew around the city of columbus
dens_obj.fence_data()

#Dbscan is the fitting method for the algorithm, it takes a proximity parameter which equals roughly 100m, and a minimum cluster mass parameter to search and identify clusters to be used as our candidate locations. See Final Report for more details.
dens_df = dens_obj.dbscan(neighborhood_dist=0.20, min_mass=60, metric="haversine")

#Remove noise from cluster, noise observations are labeled -1
dens_df = dens_df[dens_df['cluster_labels'] >= 0]

## Cluster Feature Engineering

We use the cluster labels generated by the dbscan as a grouping variable to come up with cluster specific features, which will feed into the optimization function to help drive our selection method.

In [666]:
#Get representative locations within the cluster to be used to pass through the trip distance matrix function
average_cluster_lat = dens_df.groupby('cluster_labels', as_index=False)['Lat'].mean()
average_cluster_lon = dens_df.groupby('cluster_labels', as_index=False)['Lon'].mean()

In [667]:
#Get absolute visits and unique visits by cluster, these will be used to compute the diversity of visits for a cluster
absolute_visits_by_cluster = dens_df.groupby('cluster_labels', as_index=False)['vin'].count()
unique_visits_by_cluster = dens_df.groupby('cluster_labels', as_index=False).agg({'vin': pd.Series.nunique})

In [668]:
#Clean dwell data types
dwell_raw_df = dwell_raw_df.astype({'seq':'int64'})
dwell_raw_df['start_timestamp'] = pd.to_datetime(dwell_raw_df['start_timestamp'])
dwell_raw_df['end_timestamp'] = pd.to_datetime(dwell_raw_df['end_timestamp'])
#Extract additional features
dwell_raw_df['arrival_hour'] = dwell_raw_df["start_timestamp"].dt.hour
dwell_raw_df['arrival_date'] = dwell_raw_df["start_timestamp"].dt.date
dwell_raw_df['arrival_day'] = dwell_raw_df["start_timestamp"].dt.day_name()
dwell_raw_df['trip_time'] = (dwell_raw_df['end_timestamp'] - dwell_raw_df['start_timestamp']).astype('timedelta64[m]')
#Sort and shift data to get additional features
dwell_raw_df = dwell_raw_df.sort_values(by =['vin','seq'])
dwell_raw_df["next_trip_start_timestamp"] = dwell_raw_df["start_timestamp"].shift(periods=-1)
dwell_raw_df["next_trip_seq"] = dwell_raw_df['seq'].shift(periods=-1)
dwell_raw_df["dwell_time"] = (dwell_raw_df['next_trip_start_timestamp'] - dwell_raw_df['end_timestamp']).astype('timedelta64[m]')
dwell_raw_df['seq_diff'] = dwell_raw_df['next_trip_seq'] - dwell_raw_df['seq']
dwell_raw_df['next_vin'] = dwell_raw_df['vin'].shift(periods=-1)
#Drop records that have missing timestamp values
dwell_raw_df.dropna(inplace=True)

### Get Arrival Rate

Calculate the average rate of hourly arrivals by cluster, by joining the destination data with the dwell time data. Using the `end_timestamp` from the dwell time data, we can calculate on average how many arrivals there were every hour. Because the result will not include cluster+hour combinations where there was no data, we'll back fill that using a grid expansion. We will only calculate arrival rate for the normal traffic operation hours of 6 am to 10pm. Between these hours the distribution of arrivals is consistent, and we will use this assumption to justify a poisson related method.

In [669]:
#Join Destination Data and Dwell Data
dens_dwell_df = dens_df.merge(dwell_raw_df, how="inner", on = ['seq', 'vin'])

In [670]:
#Create a summarized data frame for number of arrivals by cluster by hour
n_arrivals_df = pd.DataFrame(dens_dwell_df.groupby(by=["cluster_labels","arrival_hour"]).size())\
    .reset_index()\
        .rename(columns={0:"n_arrivals"})

In [671]:
#Create grid to inflate zeros into data
hours = [x for x in range(24)]
n_labels = dens_df['cluster_labels'].nunique()
labels = [x for x in range(n_labels)]
grid_list = [[i,j] for i in hours for j in labels]
grid_df = pd.DataFrame({"hours":[x[0] for x in grid_list], "labels": [x[1] for x in grid_list]})

In [672]:
#Use grid to expand, and back fill
expanded_arrivals_df = pd.merge(grid_df, n_arrivals_df, how="left", left_on=['hours','labels'], right_on=['arrival_hour','cluster_labels'])
expanded_arrivals_df['cluster_labels'] = expanded_arrivals_df['cluster_labels'].fillna(expanded_arrivals_df['labels'])
expanded_arrivals_df['arrival_hour'] = expanded_arrivals_df['arrival_hour'].fillna(expanded_arrivals_df['hours'])
expanded_arrivals_df['n_arrivals'] = expanded_arrivals_df['n_arrivals'].fillna(0)
expanded_arrivals_df = expanded_arrivals_df[["cluster_labels","arrival_hour","n_arrivals"]].sort_values(by = ["cluster_labels","arrival_hour"]).reset_index(drop=True)

In [673]:
#Get Arrival Rate
expanded_arrivals_df = expanded_arrivals_df[(expanded_arrivals_df['arrival_hour'] >= 5) & (expanded_arrivals_df['arrival_hour'] <= 21)]
average_hourly_arrival_by_cluster = expanded_arrivals_df.groupby("cluster_labels",as_index=False)['n_arrivals'].mean().round(2)

In [674]:
#Get Average Daily Count for visual purposes
avg_daily_base = dens_dwell_df.groupby(['cluster_labels', 'arrival_day'], as_index=False)['vin'].count()
average_daily_visits_by_cluster = avg_daily_base.groupby('cluster_labels', as_index=False)['vin'].mean().round(2)

### Get Service Rate

Calculate the average dwell time by cluster, by joining the destination data with the dwell time data. Using the `end_timestamp` from sequence t, and comparing it with the `start_timestamp` from sequence t+1 while partitioned by `vin`, we can calculated how long each vehicle stayed at each destination. However, because the data is not pristine, there are instances where the observation for sequence t+1 is not available, so estimations of dwell time in those scenarios will be removed. The `seq_diff` attribute calculated previously will be used for this. 

In [675]:
#Clean dwell_raw_df so that average_dwell_time is meaningful
dwell_clean_df = dens_dwell_df[dens_dwell_df['seq_diff']==1]
dwell_clean_df = dwell_clean_df[dwell_clean_df['vin'] == dwell_clean_df['next_vin']]
dwell_clean_df = dwell_clean_df[dwell_clean_df['dwell_time'] > 0]

In [676]:
#Get "Service Rate"
average_dwell_time_by_cluster = dwell_clean_df.groupby('cluster_labels', as_index=False)['dwell_time'].mean().round(2)

### Merge all engineered cluster features

In [677]:
#Add representative coordinate for cluster
cluster_features_df = average_cluster_lat.merge(average_cluster_lon, on="cluster_labels", how="inner")

#Add absolute number of visits for cluster
cluster_features_df = cluster_features_df.merge(absolute_visits_by_cluster, on="cluster_labels", how="inner").rename(columns={"vin": "N Visits"})

#Add absolute number of unique visits for cluster
cluster_features_df = cluster_features_df.merge(unique_visits_by_cluster, on="cluster_labels", how="inner").rename(columns={"vin": "N Unique Visits"})

#Add average hourly arrival rate for cluster
cluster_features_df = cluster_features_df.merge(average_hourly_arrival_by_cluster, on="cluster_labels", how="inner").rename(columns={"n_arrivals":"Avg Hourly N Arrivals"})

#Add average daily arrivals for cluster
cluster_features_df = cluster_features_df.merge(average_daily_visits_by_cluster, on="cluster_labels", how="inner").rename(columns={"vin":"Avg Daily N Arrivals"})

#Add average dwell time for cluster
cluster_features_df = cluster_features_df.merge(average_dwell_time_by_cluster, on="cluster_labels", how="inner")
cluster_features_df['dwell_time'] = cluster_features_df['dwell_time'] /60
cluster_features_df['dwell_time'] = cluster_features_df['dwell_time'].round(2)
cluster_features_df = cluster_features_df.rename(columns={"dwell_time":"Avg Dwell Time Hours"})

### Create features to be used for cluster utility

In [678]:
cluster_features_df['Expected Hourly N'] = cluster_features_df['Avg Hourly N Arrivals'] * cluster_features_df['Avg Dwell Time Hours']
cluster_features_df['Expected Hourly N'] = cluster_features_df['Expected Hourly N'].round(3)
cluster_features_df['Diversity'] = cluster_features_df['N Unique Visits'] / cluster_features_df['N Visits']
cluster_features_df['Diversity'] = cluster_features_df['Diversity'].round(3)
cluster_features_df['Utility'] = cluster_features_df['Expected Hourly N'] * cluster_features_df['Diversity']
cluster_features_df['Utility'] = cluster_features_df['Utility']
cluster_features_df = cluster_features_df.reset_index().rename(columns={"index":"Cluster Label"}).drop(columns={"cluster_labels"})

### kotoDistMatrix

We created a module called `kotoDistMatrix` that uses Google's Distance Matrix API to get driving distance between each of the cluster centers to use as part of one of the optimization constraints. *The code to call the Google API cannot be run or executed by the TA, since we used a personal api_key. However, the output matrix will be provided.*

In [727]:
#Import Module
import kotoDistMatrix as kdm
#Instantiate connection
kdm_obj = distance_matrix(api_key = "Google Cloud API Key goes here")
#Create distance matrix from cluster_features_df
mat = kdm_obj.create_dist_matrix(cluster_features_df)

In [ ]:
#Read in existing distance matrix
mat = np.loadtxt('data/dist_mat.csv', delimiter=',')

## Selection Optimization

### kotoOptimus

We created a module called `kotoOptimus`, which takes the distance matrix, and cluster utilities and solves for the best selection of nodes given the constraints and arbitrary budget. *See detailed information in final report*. 

In [739]:
#Import module
import kotoOptimus as ko
#Initialize optimization variables, and constants
ko_obj = ko.cluster_optimization(dataframe=cluster_features_df, mat=mat)
#Generate constraints, and minimize over objective function
selection_vector = ko_obj.optimize_cluster_selection()

Budget: 1500-1600, Run Time: 0:00:09.677325, thresh: 15, score: 1477.696763


In [ ]:
#Add selection vector as a column
cluster_features_df['Selection'] = selection_vector
#Create dataframe with only cluster selected from the optimization
cluster_selection_df = cluster_features_df[cluster_features_df['Selection']==1]

## Visualization

### kotoMap

We built a module called `kotomap`, that abstracts the interface from the Uber's open source keplerGL framework, and builds the visualization according the configuration and computed outputs from our analysis.

In [801]:
#Import
import kotoMap as km

In [803]:
#Initialize map, using custom prepared config file
km_obj = km.geo_map(config_file="data/kepler_initial_config.json")
#Run method to add default data intended for this project
    #Use add_data() method to add any additional data
km_obj.add_default_data(candidates=True, selections=True, raw_dests=True)
#Run method to save object
km_obj.export_to_html()
#Run method to display object; Note: Native displaying only works in jupyter notebook environment
    #Use arrow in top left to expand builder panel
km_obj.show()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ewe80q1g', 'type':…